In [51]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [94]:
# 데이터 불러오기
data = load_digits()
X, y = data.data, data.target
print(X[:1])
print(y[:1])

print(f'데이터 형태: {X.shape}')
print(f'타겟 형태: {y.shape}')
print(f'클래스 개수: {np.unique(y).shape[0]}')
print(f'특성 개수: {X.shape[1]}')

[[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]]
[0]
데이터 형태: (1797, 64)
타겟 형태: (1797,)
클래스 개수: 10
특성 개수: 64
16.0


In [61]:
# 훈련,학습데이터로 분할하기
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=1234,
                                                    stratify=y) # 클래스 분포 비율 유지하며 나눠줌

In [65]:
# 튜닝할 모델 준비
# Hyperparameter는 GridSearchCV가 설정함
svc_clf = SVC(random_state=1234)

In [69]:
# 하이퍼 파라미터 조합 준비
# 조합1의 수: (4*1)=4
# 조합2의 수: (4*4*1)=16
# 파라미터 조합 수: 4+16=20
param_grid = [
    # 조합1
    {
        'C': [0.1, 1, 10, 100], # 학습률
        'kernel': ['linear']
    },

    # 조합2
    {
        'C': [0.1, 1, 10, 100],
        'gamma': [0.001, 0.01, 0.1, 1],
        'kernel': ['rbf']
    },
]

In [71]:
# 교차 검증 설정(K-Fold Stratified Cross Validation)
# StratifiedKFold는 다중 클래스 분류에서 각 폴드의 클래스 분포를 원본과 비슷하게 유지시킴
cv = StratifiedKFold(n_splits=5,
                     shuffle=True,
                     random_state=1234)

In [73]:
# GridSearchCV 생성하고 학습시키기
gs = GridSearchCV(
    estimator=svc_clf, # 예측모델
    param_grid=param_grid, # 파라미터 조합
    scoring='accuracy', # 성능평가 지표
    cv=cv, # 교차검증기 설정
    n_jobs=-1 # 컴퓨터에서 가용 가능한 모든 코어 사용
)

print('시작!!!')
gs.fit(X_train, y_train)
print('끝남!!')

시작!!!
끝남!!


In [75]:
# 최적의 파라미터 조합 확인
print(gs.best_params_)

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}


In [77]:
# 최고점수 확인
print(gs.best_score_)

0.9864889647758174


In [81]:
# 여러 종류의 모델을 한번에 테스트할 수 있는데
# 최적의 모델에 best_estimator_ 속성으로 접근할 수 있음
best_model = gs.best_estimator_

# 테스트셋 예측해보기
y_pred = best_model.predict(X_test)

# 테스트셋의 정확도
print(f'{accuracy_score(y_test, y_pred):.4f}')
print(classification_report(
    y_test, y_pred,
    target_names=[str(i) for i in range(10)])
)

0.9926
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       1.00      1.00      1.00        55
           2       1.00      0.98      0.99        53
           3       1.00      0.98      0.99        55
           4       1.00      0.98      0.99        54
           5       1.00      0.98      0.99        55
           6       1.00      1.00      1.00        54
           7       0.96      1.00      0.98        54
           8       0.98      1.00      0.99        52
           9       0.98      1.00      0.99        54

    accuracy                           0.99       540
   macro avg       0.99      0.99      0.99       540
weighted avg       0.99      0.99      0.99       540



In [98]:
X_3 = np.array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16., 16., 16.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,  0.,  0.,  0.,  0.,  0.,
        16., 16., 16., 16., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,
         0.,  0.,  0.,  0.,  0.,  0., 16., 16.,  0.,  0.,  0., 16., 16.,
        16., 16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])
y_pred_3 = best_model.predict(X_3)
y_pred_3

array([1])

In [104]:
# 이미지 데이터 시각화하는 코드
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
y[3]
num3 = X[3]

num3 = num3.reshape((8,8))
print(num3.shape)

scaler = MinMaxScaler(feature_range=(0,255))
scaled_num_3 = scaler.fit_transform(num3)

img = Image.fromarray(scaled_num_3)
img.show()

(8, 8)
